In [ ]:
class Const(Expr):
    def __init__(self, value):
        self.value = value
    
    def __call__(self, **context):
        return self.value
    
    def d(self, wrt):
        return Const(0)
    
    def __str__(self) -> str:
        return str(self.value)

class Var(Expr):
    def __init__(self, name):
        self.name = name
    
    def __call__(self, **context):
        return context[self.name]
    
    def d(self, wrt):

        if isinstance(wrt, Var):
            wrt = wrt.name

        if self.name == wrt:
            return Const(1)
        return Const(0)
    
    def __str__(self):
        return self.name   

NameError: name 'Expr' is not defined

In [ ]:
class BinOp(Expr):
    def __init__(self, expr1: Expr, expr2: Expr) -> None:
        self.expr1, self.expr2 = expr1, expr2

class Neg(Expr):
    def __init__(self, expr):
        self.expr = expr
    
    def __call__(self, **context):
        return -self.expr(**context)
    
    def d(self, wrt):
        return Neg(self.expr.d(wrt))
    
    def __str__(self):
        return f'(- {self.expr})'

In [ ]:
class Sum(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)

    def d(self, wrt):
        '''
        d[f(x) + g(x)]/dx = f'(x) + g'(x)
        '''
        return Sum(self.expr1.d(wrt), self.expr2.d(wrt))

    def __str__(self):
        return f'(+ {self.expr1} {self.expr2})'

class Product(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)
    
    def d(self, wrt):
        '''
        d[f(x)*g(x)]/dx = f(x)*g'(x) + f'(x)*g(x)
        '''
        return Sum(
            Product(self.expr1, self.expr2.d(wrt)),
            Product(self.expr1.d(wrt), self.expr2)
            )

    def __str__(self):
        return f'(* {self.expr1} {self.expr2})'

class Fraction(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) / self.expr2(**context)
    
    def d(self, wrt):
        '''
        d[f(x)/g(x)]/dx = [f'(x)g(x) - f(x)g'(x)]/[g(x)*g(x)]
        '''
        return Fraction(
            Sum(
                Product(self.expr1.d(wrt), self.expr2),
                Neg(Product(self.expr1, self.expr2.d(wrt)))
            ),
            Product(self.expr2, self.expr2)
        )
    
    def __str__(self):
        return f'(/ {self.expr1} {self.expr2})'

In [ ]:
class Power(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) ** self.expr2(**context)

    def d(self, wrt):
        '''
        d[f(x)^n] = (n*f(x)^(n-1))*f'(x)
        '''
        return Product(
            Product(self.expr2, # n * (f(x) ^ (n-1))
                    Power(self.expr1, # f(x) ^ (n-1)
                          Sum(self.expr2, Neg(Const(1))))), # n-1
            self.expr1.d(wrt)
        )
    
    def __str__(self):
        return f'(** {self.expr1} {self.expr2})'

In [ ]:
class Expr:
    def __call__(self, **context):
        pass
    
    def d(self, wrt):
        pass

    def __neg__(e):
        return Neg(e)

    def __pos__(e):
        return e
    
    def __add__(e1, e2):
        return Sum(e1, e2)

    def __sub__(e1, e2):
        return Sum(e1, Neg(e2))

    def __mul__(e1, e2):
        return Product(e1, e2)

    def __truediv__(e1, e2):
        return Fraction(e1, e2)
    
    def __pow__(e1, e2):
        return Power(e1, e2)

In [ ]:
class Expr:
    def __sub__(self, other):
        return Sum(self, Neg(other))

    def __add__(self, other):
        return Sum(self, other)

    def __mul__(self, other):
        return Product(self, other)

    def __truediv__(self, other):
        return Fraction(self, other)

    def __pow__(self, other):
        return Power(self, other)


class Const(Expr):
    def __init__(self, value):
        self.value = value
    
    def __call__(self, **context):
        return self.value
    
    def d(self, wrt):
        return Const(0)
    
    def __str__(self) -> str:
        return str(self.value)


class Var(Expr):
    def __init__(self, name):
        self.name = name
    
    def __call__(self, **context):
        return context[self.name]
    
    def d(self, wrt):
        if isinstance(wrt, Var):
            wrt = wrt.name
        return Const(1) if self.name == wrt else Const(0)
    
    def __str__(self):
        return self.name


C = Const
V = Var


class BinOp(Expr):
    def __init__(self, expr1: Expr, expr2: Expr) -> None:
        self.expr1, self.expr2 = expr1, expr2


class Neg(Expr):
    def __init__(self, expr):
        self.expr = expr
    
    def __call__(self, **context):
        return -self.expr(**context)
    
    def d(self, wrt):
        return Neg(self.expr.d(wrt))
    
    def __str__(self):
        return f'(- {self.expr})'


class Sum(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) + self.expr2(**context)

    def d(self, wrt):
        return Sum(self.expr1.d(wrt), self.expr2.d(wrt))

    def __str__(self):
        return f'(+ {self.expr1} {self.expr2})'


class Product(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) * self.expr2(**context)
    
    def d(self, wrt):
        return Sum(
            Product(self.expr1, self.expr2.d(wrt)),
            Product(self.expr1.d(wrt), self.expr2)
        )

    def __str__(self):
        return f'(* {self.expr1} {self.expr2})'


class Fraction(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) / self.expr2(**context)
    
    def d(self, wrt):
        return Fraction(
            Sum(
                Product(self.expr1.d(wrt), self.expr2),
                Neg(Product(self.expr1, self.expr2.d(wrt)))
            ),
            Product(self.expr2, self.expr2)
        )
    
    def __str__(self):
        return f'(/ {self.expr1} {self.expr2})'


class Power(BinOp):
    def __call__(self, **context):
        return self.expr1(**context) ** self.expr2(**context)

    def d(self, wrt):
        return Product(
            Product(
                self.expr2,
                Power(self.expr1, Sum(self.expr2, Neg(Const(1))))
            ),
            self.expr1.d(wrt)
        )
    
    def __str__(self):
        return f'(** {self.expr1} {self.expr2})'


print(((C(1) - V("x")) ** C(3) + V("x"))(x=12))
print(((C(1) + V("x")) ** C(2)).d("x")(x=3))


-1319
8
